# Title
[]()

In [2]:
import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
import time
import re
from itertools import product
import openai

from response_processing import *
from article_processing import create_text_dict_from_folder
import traceback
from silvhua import *

In [38]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
# from wrangling import filter_df_all_conditions, filter_df_any_condition

# Original scripts

In [ ]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, 
    temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the values are pandas dataframes containing the summaries for each text ID

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='batch_Chaining_summaries',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_instance_to_dict(
                    chaining_bot_dict[iteration_id], 
                    description=f'batch_Chaining_attributes',
                    pickle_path=pickle_path, json_path=json_path
                    )
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


# Set up

In [2]:
prep_step = [
    "Think about why this might be relevant for the audience in the grand scheme of things.\
    \nIdentify 1 or 2 key concepts from this article that would make interesting or helpful health content. \
    Exclude details that do not add value to the audience.\
    \nBased on the key concepts from the previous steps, extract the key points and numerical descriptors to",
]

summarize_task = [
    "summarize for a LinkedIn post.",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create an Instagram post without hashtags.",
]
edit_task = [
    "\nIf applicable, include a brief description of the research participants, such as age and sex.\
    Otherwise, you can skip this step.\
    \nEvaluate whether or not your writing may be confusing. \
    \nIf so, re-write it so it is clear. Otherwise, keep it the same. \
    \nCreate a headline to hook the audience.\
    \nReturn your response as the headline followed by the final version of the summary, \
    separated by a blank line.",
]

system_role = "You are a journalist writing content based on science research articles."
prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
    columns=['prep_step', 'summarize_task', 'edit_task'])
prompts_df

,prep_step,summarize_task,edit_task
0,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o..."


In [5]:
# Create text dictionary
folder_path = '../text/2023-06-11' # ** UPDATE REQUIRED**

encoding='ISO-8859-1'
subset=None

text_dict = create_text_dict_from_folder(folder_path, encoding=encoding, subset=subset)


chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()
chain_results_dict = dict()
qna_dict = dict()

In [7]:
(text_dict)

{1: 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial\nDiscussion\nThis nutritional approach using high calcium and high protein dairy foods to increase calcium and protein intakes in institutionalised older adults replete in vitamin D was associated with a 33% reduction in risk of fractures of any type, a 46% reduction in risk of hip fractures, and an 11% reduction in risk of falls relative to controls. We found no group difference in all cause mortality.\nMost interventions aimed at reducing fracture risk target a drug therapy to people with osteoporosis because they are at high risk of fracture. This approach confers a large benefit to the individual and does so cost effectively, because few people need to be treated to avert one event. However, averting fractures in small numbers of people at high risk does not reduce the burden of fractures in the community.\nThe population burden of

# Iteration 1

## 1.10 Update messages and class attributes

In [44]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        # try:
        #     response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        # except Exception as error:
        #     exc_type, exc_obj, tb = sys.exc_info()
        #     f = tb.tb_frame
        #     lineno = tb.tb_lineno
        #     filename = f.f_code.co_filename
        #     print("An error occurred on line", lineno, "in", filename, ":", error)
        #     print('\t**API request failed for `.summarize()`**')
        #     return self.qna
        # try:
        #     for index, choice in enumerate(response.choices):
        #         self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        #     self.qna.setdefault('summary', [])
        #     self.qna['summary'].extend([value for value in self.summaries_dict.values()])
        #     self.summaries_dict['prep_step'] = prep_step
        #     self.summaries_dict['task'] = task
        #     self.summaries_dict['prompt'] = full_task
        #     return self.qna
        # except Exception as error:
        #     exc_type, exc_obj, tb = sys.exc_info()
        #     f = tb.tb_frame
        #     lineno = tb.tb_lineno
        #     filename = f.f_code.co_filename
        #     print("An error occurred on line", lineno, "in", filename, ":", error)
        #     print('\t**Error with response parsing**')
        #     return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the values are pandas dataframes containing the summaries for each text ID

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
    return chaining_bot_dict

    #     try:
    #         updated_qna_dict = (temp_qna_dict[key])
    #     except Exception as error:
    #         exc_type, exc_obj, tb = sys.exc_info()
    #         f = tb.tb_frame
    #         lineno = tb.tb_lineno
    #         filename = f.f_code.co_filename
    #         print("An error occurred on line", lineno, "in", filename, ":", error)
    #         print('Error concatenating prompts DataFrame')
    #         return temp_qna_dict, chaining_bot_dict
    #     qna_dfs_list.append(updated_qna_dict)
    # try:
    #     qna_dict[iteration_id] = pd.concat([
    #         pd.DataFrame(
    #             data, index=[choice for choice in range(1, len(data['summary'])+1)]
    #         ) for dictionary in qna_dfs_list for data in dictionary.values()
    #     ])
    #     qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
    #     print('DataFrame shape:', qna_dict[iteration_id].shape)
    #     if save_outputs:
    #         try:
    #             save_output(
    #                 qna_dict[iteration_id], description='batch_Chaining_summaries',
    #                 csv_path=csv_path, pickle_path=pickle_path)
    #             save_instance_to_dict(
    #                 chaining_bot_dict[iteration_id], 
    #                 description=f'batch_Chaining_attributes',
    #                 pickle_path=pickle_path, json_path=json_path
    #                 )
    #         except:
    #             print('[prompt_chaining_dict()] Unable to save outputs')
    # except Exception as error:
    #     exc_type, exc_obj, tb = sys.exc_info()
    #     f = tb.tb_frame
    #     lineno = tb.tb_lineno
    #     filename = f.f_code.co_filename
    #     print("An error occurred on line", lineno, "in", filename, ":", error)
    #     qna_dict[iteration_id] = qna_dfs_list
    #     print('Error creating DataFrame; dictionary returned instead')
    # return qna_dict, chaining_bot_dict


# Set parameters
iteration_id = 1
n_choices = 2
pause_per_request=0
summary_iteration_id = iteration_id
chatbot_id = iteration_id

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chatbot_dict,
    system_role=system_role, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id
    )


**Text #1 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	...Success!
**Text #2 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	...Success!


In [47]:
chaining_dict

{1: {'text1_prompt00': <__main__.Chaining at 0x198fa2e0d90>,
  'text2_prompt00': <__main__.Chaining at 0x198fa2e1450>}}

In [45]:
chaining_dict[1]['text1_prompt00'].qna

{'article_title': 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial',
 'system_role': 'You are a helpful assistant.',
 'model': 'gpt-3.5-turbo',
 'text': 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial\nDiscussion\nThis nutritional approach using high calcium and high protein dairy foods to increase calcium and protein intakes in institutionalised older adults replete in vitamin D was associated with a 33% reduction in risk of fractures of any type, a 46% reduction in risk of hip fractures, and an 11% reduction in risk of falls relative to controls. We found no group difference in all cause mortality.\nMost interventions aimed at reducing fracture risk target a drug therapy to people with osteoporosis because they are at high risk of fracture. This approach confers a large benefit to the i

In [46]:
print(chaining_dict[1]['text1_prompt00'].qna['full summarization task'])

Think about why this might be relevant for the audience in the grand scheme of things.    
Identify 1 or 2 key concepts from this article that would make interesting or helpful health content.     Exclude details that do not add value to the audience.    
Based on the key concepts from the previous steps, extract the key points and numerical descriptors to summarize for a LinkedIn post. 
If applicable, include a brief description of the research participants, such as age and sex.    Otherwise, you can skip this step.    
Evaluate whether or not your writing may be confusing.     
If so, re-write it so it is clear. Otherwise, keep it the same.     
Create a headline to hook the audience.    
Return your response as the headline followed by the final version of the summary,     separated by a blank line.


In [48]:
vars(chaining_dict[1]['text1_prompt00'])

{'text': 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial\nDiscussion\nThis nutritional approach using high calcium and high protein dairy foods to increase calcium and protein intakes in institutionalised older adults replete in vitamin D was associated with a 33% reduction in risk of fractures of any type, a 46% reduction in risk of hip fractures, and an 11% reduction in risk of falls relative to controls. We found no group difference in all cause mortality.\nMost interventions aimed at reducing fracture risk target a drug therapy to people with osteoporosis because they are at high risk of fracture. This approach confers a large benefit to the individual and does so cost effectively, because few people need to be treated to avert one event. However, averting fractures in small numbers of people at high risk does not reduce the burden of fractures in the community.\nThe population burd

## 1.11 Send to ChatGPT

In [8]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        qna_dict: dict
            A dictionary containing the results of the summarization process. The keys of the dictionary are the iteration IDs and the values are pandas dataframes containing the summaries for each text ID

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
    return chaining_bot_dict

    #     try:
    #         updated_qna_dict = (temp_qna_dict[key])
    #     except Exception as error:
    #         exc_type, exc_obj, tb = sys.exc_info()
    #         f = tb.tb_frame
    #         lineno = tb.tb_lineno
    #         filename = f.f_code.co_filename
    #         print("An error occurred on line", lineno, "in", filename, ":", error)
    #         print('Error concatenating prompts DataFrame')
    #         return temp_qna_dict, chaining_bot_dict
    #     qna_dfs_list.append(updated_qna_dict)
    # try:
    #     qna_dict[iteration_id] = pd.concat([
    #         pd.DataFrame(
    #             data, index=[choice for choice in range(1, len(data['summary'])+1)]
    #         ) for dictionary in qna_dfs_list for data in dictionary.values()
    #     ])
    #     qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
    #     print('DataFrame shape:', qna_dict[iteration_id].shape)
    #     if save_outputs:
    #         try:
    #             save_output(
    #                 qna_dict[iteration_id], description='batch_Chaining_summaries',
    #                 csv_path=csv_path, pickle_path=pickle_path)
    #             save_instance_to_dict(
    #                 chaining_bot_dict[iteration_id], 
    #                 description=f'batch_Chaining_attributes',
    #                 pickle_path=pickle_path, json_path=json_path
    #                 )
    #         except:
    #             print('[prompt_chaining_dict()] Unable to save outputs')
    # except Exception as error:
    #     exc_type, exc_obj, tb = sys.exc_info()
    #     f = tb.tb_frame
    #     lineno = tb.tb_lineno
    #     filename = f.f_code.co_filename
    #     print("An error occurred on line", lineno, "in", filename, ":", error)
    #     qna_dict[iteration_id] = qna_dfs_list
    #     print('Error creating DataFrame; dictionary returned instead')
    # return qna_dict, chaining_bot_dict



# Set parameters
iteration_id = 1
n_choices = 2
pause_per_request=0
summary_iteration_id = iteration_id
chatbot_id = iteration_id

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chatbot_dict,
    system_role=system_role, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id
    )


**Text #1 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!


### Check results

In [12]:
chaining_dict

{1: {'text1_prompt00': <__main__.Chaining at 0x22979a07b90>,
  'text2_prompt00': <__main__.Chaining at 0x229799d4250>}}

In [14]:
next(iter(chaining_dict[1]))

'text1_prompt00'

In [18]:
chaining_dict[1][next(iter(chaining_dict[1]))]

In [19]:
vars(chaining_dict[1][next(iter(chaining_dict[1]))])

{'text': 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial\nDiscussion\nThis nutritional approach using high calcium and high protein dairy foods to increase calcium and protein intakes in institutionalised older adults replete in vitamin D was associated with a 33% reduction in risk of fractures of any type, a 46% reduction in risk of hip fractures, and an 11% reduction in risk of falls relative to controls. We found no group difference in all cause mortality.\nMost interventions aimed at reducing fracture risk target a drug therapy to people with osteoporosis because they are at high risk of fracture. This approach confers a large benefit to the individual and does so cost effectively, because few people need to be treated to avert one event. However, averting fractures in small numbers of people at high risk does not reduce the burden of fractures in the community.\nThe population burd

## `sample_Chaining_attr` function

In [29]:
from pprint import pprint

def sample_Chaining_attr(chaining_dict=chaining_dict, iteration_id=iteration_id):
    return (vars(chaining_dict[iteration_id][next(iter(chaining_dict[iteration_id]))]))

# sample_Chaining_attr()

In [28]:
sample_Chaining_attr()['summaries_dict']

{'response_01': 'High calcium and high protein dairy foods may reduce risk of fractures and falls in institutionalized older adults. This nutritional approach was associated with a 33% reduction in risk of fractures of any type, a 46% reduction in risk of hip fractures, and an 11% reduction in risk of falls relative to controls. The study intentionally targeted a cohort at high risk for fracture in whom low calcium and protein intakes were common and compliance was optimized by supervised provision and consumption of the foods. Milk consumption did not differ between the intervention and control groups. This research could be helpful for healthcare professionals working with older adults in residential care. \n\nHeadline: Nutritional Approach May Reduce Risk of Fractures and Falls in Institutionalized Older Adults',
 'response_02': 'High calcium and protein dairy foods may reduce risk of fractures and falls in older adults\n\nA cluster randomized controlled trial found that older adult

In [29]:
sample_Chaining_attr()['qna']

{'article_title': 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial',
 'system_role': 'You are a helpful assistant.',
 'model': 'gpt-3.5-turbo',
 'text': 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial\nDiscussion\nThis nutritional approach using high calcium and high protein dairy foods to increase calcium and protein intakes in institutionalised older adults replete in vitamin D was associated with a 33% reduction in risk of fractures of any type, a 46% reduction in risk of hip fractures, and an 11% reduction in risk of falls relative to controls. We found no group difference in all cause mortality.\nMost interventions aimed at reducing fracture risk target a drug therapy to people with osteoporosis because they are at high risk of fracture. This approach confers a large benefit to the i

In [60]:
print(sample_Chaining_attr()['qna']['full summarization task'])

Think about why this might be relevant for the audience in the grand scheme of things.    
Identify 1 or 2 key concepts from this article that would make interesting or helpful health content.     Exclude details that do not add value to the audience.    
Based on the key concepts from the previous steps, extract the key points and numerical descriptors to summarize for a LinkedIn post. 
If applicable, include a brief description of the research participants, such as age and sex.    Otherwise, you can skip this step.    
Evaluate whether or not your writing may be confusing.     
If so, re-write it so it is clear. Otherwise, keep it the same.     
Create a headline to hook the audience.    
Return your response as the headline followed by the final version of the summary,     separated by a blank line.


## 1.12 Create summaries dataframe

In [47]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
    return chaining_bot_dict

def create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    ):
    """
    """
    dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(names=['choice']))
    
    return dfs_list
    qna_df = pd.concat(dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    qna_dict[iteration_id] = qna_df
    return qna_dict

    #     try:
    #         updated_qna_dict = (temp_qna_dict[key])
    #     except Exception as error:
    #         exc_type, exc_obj, tb = sys.exc_info()
    #         f = tb.tb_frame
    #         lineno = tb.tb_lineno
    #         filename = f.f_code.co_filename
    #         print("An error occurred on line", lineno, "in", filename, ":", error)
    #         print('Error concatenating prompts DataFrame')
    #         return temp_qna_dict, chaining_bot_dict
    #     qna_dfs_list.append(updated_qna_dict)
    # try:
    #     qna_dict[iteration_id] = pd.concat([
    #         pd.DataFrame(
    #             data, index=[choice for choice in range(1, len(data['summary'])+1)]
    #         ) for dictionary in qna_dfs_list for data in dictionary.values()
    #     ])
    #     qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
    #     print('DataFrame shape:', qna_dict[iteration_id].shape)
    #     if save_outputs:
    #         try:
    #             save_output(
    #                 qna_dict[iteration_id], description='batch_Chaining_summaries',
    #                 csv_path=csv_path, pickle_path=pickle_path)
    #             save_instance_to_dict(
    #                 chaining_bot_dict[iteration_id], 
    #                 description=f'batch_Chaining_attributes',
    #                 pickle_path=pickle_path, json_path=json_path
    #                 )
    #         except:
    #             print('[prompt_chaining_dict()] Unable to save outputs')
    # except Exception as error:
    #     exc_type, exc_obj, tb = sys.exc_info()
    #     f = tb.tb_frame
    #     lineno = tb.tb_lineno
    #     filename = f.f_code.co_filename
    #     print("An error occurred on line", lineno, "in", filename, ":", error)
    #     qna_dict[iteration_id] = qna_dfs_list
    #     print('Error creating DataFrame; dictionary returned instead')
    # return qna_dict, chaining_bot_dict



# Set parameters
iteration_id = 1
n_choices = 2
pause_per_request=0
summary_iteration_id = iteration_id
chatbot_id = iteration_id

# # Create initial summaries
# chaining_dict = batch_summarize_chain(
#     text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chatbot_dict,
#     system_role=system_role, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id
#     )

qna_dict = dict()
qna_dict = create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    )

Processing text1_prompt00...
Processing text2_prompt00...


In [43]:
chatbot_dict

{1: {'text1_prompt00': <__main__.Chaining at 0x22979a07b90>,
  'text2_prompt00': <__main__.Chaining at 0x229799d4250>}}

In [49]:
qna_dict

[   choice  \
 0       0   
 1       1   
 
                                                                                          article_title  \
 0  Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...   
 1  Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...   
 
                     system_role          model  \
 0  You are a helpful assistant.  gpt-3.5-turbo   
 1  You are a helpful assistant.  gpt-3.5-turbo   
 
                                                                                                   text  \
 0  Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...   
 1  Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...   
 
                                                                                              prep step  \
 0  Think about why this might be relevant for t

In [48]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,edit task,full summarization task,summary
0,0,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,You are a helpful assistant.,gpt-3.5-turbo,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and sex. ...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Headline: Weight Stigma Associated with Poorer Health Behaviors, Independent of BMI\n\nKey conce..."
1,1,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,You are a helpful assistant.,gpt-3.5-turbo,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and sex. ...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Weight stigma may undermine healthy behaviors, according to a study in the International Journal..."


## 1.13

In [58]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
    return chaining_bot_dict

def create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    ):
    """
    """
    dfs_list = []
    chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        print(f'Processing {chatbot_key}...')
        # dfs_list.append(pd.DataFrame(chatbot_dict[chatbot_id][chatbot_key].qna).reset_index(names=['choice']))
        dfs_list.append(pd.DataFrame(
            chatbot_dict[chatbot_id][chatbot_key].qna, 
            index=[choice for choice in range(1, len(chatbot_dict[chatbot_id][chatbot_key].qna['summary'])+1)])
            )
    
    qna_df = pd.concat(dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    qna_dict[iteration_id] = qna_df
    return qna_dict



# Set parameters
iteration_id = 1.13
n_choices = 2
pause_per_request=0
summary_iteration_id = iteration_id
chatbot_id = iteration_id

# # Create initial summaries
# chaining_dict = batch_summarize_chain(
#     text_dict, prep_step, summarize_task, edit_task, temp_qna_dict, chatbot_dict,
#     system_role=system_role, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id
#     )

chatbot_id = 1
qna_dict = dict()
qna_dict = create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
    )
qna_dict[iteration_id]

Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (4, 9)
Original summaries Dataframe columns: Index(['article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'edit task', 'full summarization task',
       'summary'],
      dtype='object')


,article_title,system_role,model,text,prep step,summarization task,edit task,full summarization task,summary
1,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,You are a helpful assistant.,gpt-3.5-turbo,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and sex. ...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,High calcium and high protein dairy foods may reduce risk of fractures and falls in institutiona...
2,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,You are a helpful assistant.,gpt-3.5-turbo,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and sex. ...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,High calcium and protein dairy foods may reduce risk of fractures and falls in older adults\n\nA...
1,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,You are a helpful assistant.,gpt-3.5-turbo,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and sex. ...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Headline: Weight Stigma Associated with Poorer Health Behaviors, Independent of BMI\n\nKey conce..."
2,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,You are a helpful assistant.,gpt-3.5-turbo,Weight stigma and health behaviors: evidence from the Eating in America Study. International Jou...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and sex. ...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Weight stigma may undermine healthy behaviors, according to a study in the International Journal..."


In [54]:
qna_dict.keys()

dict_keys([1])

## 1.14 save the API response

In [7]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, csv_path=folder_path, pickle_path=folder_path, json_path=folder_path
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature, system_role=system_role)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_output(
                qna_dict[iteration_id], description='batch_Chaining_responses',
                csv_path=csv_path, pickle_path=pickle_path)
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[prompt_chaining_dict()] Unable to save outputs')

    return chaining_bot_dict

def create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    ):
    """
    Create DataFrame from initial ChatGPT summaries.
    """
    dfs_list = []
    chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        print(f'Processing {chatbot_key}...')
        # dfs_list.append(pd.DataFrame(chatbot_dict[chatbot_id][chatbot_key].qna).reset_index(names=['choice']))
        dfs_list.append(pd.DataFrame(
            chatbot_dict[chatbot_id][chatbot_key].qna, 
            index=[choice for choice in range(1, len(chatbot_dict[chatbot_id][chatbot_key].qna['summary'])+1)])
            )
    
    qna_df = pd.concat(dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    qna_dict[iteration_id] = qna_df
    return qna_dict



# Set parameters
iteration_id = 1.14
n_choices = 2
pause_per_request=0
chatbot_id = iteration_id

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id
    )

# chatbot_id = 1
qna_dict = create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
    )
qna_dict[iteration_id]

**Text #1 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (4, 9)
Original summaries Dataframe columns: Index(['article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'edit task', 'full summarization task',
       'summary'],
      dtype='object')


,article_title,system_role,model,text,prep step,summarization task,edit task,full summarization task,summary
1,Effect of dietary sources of calcium and prote...,You are a journalist writing content based on ...,gpt-3.5-turbo,Effect of dietary sources of calcium and prote...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Nutritional approach reduces fractures and fal...
2,Effect of dietary sources of calcium and prote...,You are a journalist writing content based on ...,gpt-3.5-turbo,Effect of dietary sources of calcium and prote...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,High calcium and high protein dairy foods may ...
1,Weight stigma and health behaviors: evidence f...,You are a journalist writing content based on ...,gpt-3.5-turbo,Weight stigma and health behaviors: evidence f...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Headline: Weight Stigma Linked to Poor Health ...
2,Weight stigma and health behaviors: evidence f...,You are a journalist writing content based on ...,gpt-3.5-turbo,Weight stigma and health behaviors: evidence f...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Headline: Weight Stigma Associated with Poor H...


In [8]:
chaining_dict

{1.14: {'text1_prompt00': <__main__.Chaining at 0x17bd1300590>,
  'text2_prompt00': <__main__.Chaining at 0x17bce185ad0>}}

In [11]:
save_outputs = True
pickle_path=folder_path
json_path=folder_path
if save_outputs:
    try:
        save_instance_to_dict(
            chaining_dict[iteration_id], 
            description=f'batch_Chaining_responses',
            pickle_path=pickle_path, json_path=json_path
            )
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        file = f.f_code.co_filename
        print(f'An error occurred on line {lineno} in {file}: {error}')
        print('[prompt_chaining_dict()] Unable to save outputs')


text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	n_previous_prompts
text2_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	n_previous_prompts
File saved:  ../text/2023-06-11/batch_Chaining_responses_2023-06-12_1100.sav
Time completed: 2023-06-12 11:00:13.074088
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00', 'text2_prompt00'])
Object saved as JSON: batch_Chaining_responses_2023-06-12_1100.json


## 1.15

In [18]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, csv_path=folder_path, pickle_path=folder_path, json_path=folder_path
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature, system_role=system_role)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_initial',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[batch_summarize_chain()] Unable to save API response')

    return chaining_bot_dict

def create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    ):
    """
    Create DataFrame from initial ChatGPT summaries.
    """
    dfs_list = []
    chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        print(f'Processing {chatbot_key}...')
        # dfs_list.append(pd.DataFrame(chatbot_dict[chatbot_id][chatbot_key].qna).reset_index(names=['choice']))
        dfs_list.append(pd.DataFrame(
            chatbot_dict[chatbot_id][chatbot_key].qna, 
            index=[choice for choice in range(1, len(chatbot_dict[chatbot_id][chatbot_key].qna['summary'])+1)])
            )
    
    qna_df = pd.concat(dfs_list).reset_index(names=['choice'])
    # Move 'choice' to second column
    columns = qna_df.columns.tolist()
    columns.remove('choice')
    columns.insert(1, 'choice')
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    qna_dict[iteration_id] = qna_df[columns]
    return qna_dict

def spreadsheet_columns(qna_dict, chatbot_dict, iteration_id, chatbot_id=None,
    save=False, filename=None, csv_path=folder_path, pickle_path=folder_path
    ):
    """
    Update column names to include corresponding column in a spreadsheet (e.g. A, B, C)
    """
    qna_dict = create_qna_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
        )
    spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
    qna_dict[iteration_id].columns = [
        f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(qna_dict[iteration_id].columns)
        ]
    if save:
        description = filename if filename else 'batch_Chaining_summaries_initial'
        try:
            save_output(
                qna_dict[iteration_id], description=description, append_version=True,
                csv_path=csv_path, pickle_path=pickle_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[spreadsheet_columns()] Unable to save original summaries DataFrame')
    return qna_dict


# Set parameters
iteration_id = 1.15
n_choices = 2
pause_per_request=0
chatbot_id = iteration_id

# # Create initial summaries
# chaining_dict = batch_summarize_chain(
#     text_dict, prep_step, summarize_task, edit_task, chatbot_dict,
#     system_role=system_role, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id, save_outputs=True
#     )

# chatbot_id = 1.15
# qna_dict = create_qna_df(
#     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
#     )
chatbot_id = 1.14
save = True
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )
qna_dict[iteration_id]

Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (4, 10)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'edit task', 'full summarization task',
       'summary'],
      dtype='object')
File saved:  ../text/2023-06-11/batch_Chaining_summaries_initial_2023-06-12_1139.sav
Time completed: 2023-06-12 11:39:43.138043
	Object saved as pickle
File saved:  ../text/2023-06-11/batch_Chaining_summaries_initial_2023-06-12_1139.csv
Time completed: 2023-06-12 11:39:43.176313
	DataFrame saved as CSV


,B: article_title,C: choice,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: edit task,J: full summarization task,K: summary
0,Effect of dietary sources of calcium and prote...,1,You are a journalist writing content based on ...,gpt-3.5-turbo,Effect of dietary sources of calcium and prote...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Nutritional approach reduces fractures and fal...
1,Effect of dietary sources of calcium and prote...,2,You are a journalist writing content based on ...,gpt-3.5-turbo,Effect of dietary sources of calcium and prote...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,High calcium and high protein dairy foods may ...
2,Weight stigma and health behaviors: evidence f...,1,You are a journalist writing content based on ...,gpt-3.5-turbo,Weight stigma and health behaviors: evidence f...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Headline: Weight Stigma Linked to Poor Health ...
3,Weight stigma and health behaviors: evidence f...,2,You are a journalist writing content based on ...,gpt-3.5-turbo,Weight stigma and health behaviors: evidence f...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Headline: Weight Stigma Associated with Poor H...


In [22]:
print(qna_dict[iteration_id]['I: edit task'].str.strip())

0    If applicable, include a brief description of ...
1    If applicable, include a brief description of ...
2    If applicable, include a brief description of ...
3    If applicable, include a brief description of ...
Name: I: edit task, dtype: object


In [24]:
print(qna_dict[iteration_id].loc[0, 'I: edit task'].strip())

If applicable, include a brief description of the research participants, such as age and sex.    Otherwise, you can skip this step.    
Evaluate whether or not your writing may be confusing.     
If so, re-write it so it is clear. Otherwise, keep it the same.     
Create a headline to hook the audience.    
Return your response as the headline followed by the final version of the summary,     separated by a blank line.


## 1.15 

In [ ]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, csv_path=folder_path, pickle_path=folder_path, json_path=folder_path
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature, system_role=system_role)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_initial',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[batch_summarize_chain()] Unable to save API response')

    return chaining_bot_dict

def create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    ):
    """
    Create DataFrame from initial ChatGPT summaries.
    """
    dfs_list = []
    chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        print(f'Processing {chatbot_key}...')
        # dfs_list.append(pd.DataFrame(chatbot_dict[chatbot_id][chatbot_key].qna).reset_index(names=['choice']))
        dfs_list.append(pd.DataFrame(
            chatbot_dict[chatbot_id][chatbot_key].qna, 
            index=[choice for choice in range(1, len(chatbot_dict[chatbot_id][chatbot_key].qna['summary'])+1)])
            )
    
    qna_df = pd.concat(dfs_list).reset_index(names=['choice'])
    # Move 'choice' to second column
    columns = qna_df.columns.tolist()
    columns.remove('choice')
    columns.insert(1, 'choice')
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    qna_dict[iteration_id] = qna_df[columns]
    return qna_dict

def spreadsheet_columns(qna_dict, chatbot_dict, iteration_id, chatbot_id=None,
    save=False, filename=None, csv_path=folder_path, pickle_path=folder_path
    ):
    """
    Update column names to include corresponding column in a spreadsheet (e.g. A, B, C)
    """
    qna_dict = create_qna_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
        )
    spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
    qna_dict[iteration_id].columns = [
        f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(qna_dict[iteration_id].columns)
        ]
    if save:
        description = filename if filename else 'batch_Chaining_summaries_initial'
        try:
            save_output(
                qna_dict[iteration_id], description=description, append_version=True,
                csv_path=csv_path, pickle_path=pickle_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[spreadsheet_columns()] Unable to save original summaries DataFrame')
    return qna_dict


# Set parameters
iteration_id = 1.15
n_choices = 2
pause_per_request=0
chatbot_id = iteration_id

# # Create initial summaries
# chaining_dict = batch_summarize_chain(
#     text_dict, prep_step, summarize_task, edit_task, chatbot_dict,
#     system_role=system_role, 
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id, save_outputs=True
#     )

# chatbot_id = 1.15
# qna_dict = create_qna_df(
#     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
#     )
chatbot_id = 1.14
save = True
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )
qna_dict[iteration_id]

Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (4, 10)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'edit task', 'full summarization task',
       'summary'],
      dtype='object')
File saved:  ../text/2023-06-11/batch_Chaining_summaries_initial_2023-06-12_1139.sav
Time completed: 2023-06-12 11:39:43.138043
	Object saved as pickle
File saved:  ../text/2023-06-11/batch_Chaining_summaries_initial_2023-06-12_1139.csv
Time completed: 2023-06-12 11:39:43.176313
	DataFrame saved as CSV


,B: article_title,C: choice,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: edit task,J: full summarization task,K: summary
0,Effect of dietary sources of calcium and prote...,1,You are a journalist writing content based on ...,gpt-3.5-turbo,Effect of dietary sources of calcium and prote...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Nutritional approach reduces fractures and fal...
1,Effect of dietary sources of calcium and prote...,2,You are a journalist writing content based on ...,gpt-3.5-turbo,Effect of dietary sources of calcium and prote...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,High calcium and high protein dairy foods may ...
2,Weight stigma and health behaviors: evidence f...,1,You are a journalist writing content based on ...,gpt-3.5-turbo,Weight stigma and health behaviors: evidence f...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Headline: Weight Stigma Linked to Poor Health ...
3,Weight stigma and health behaviors: evidence f...,2,You are a journalist writing content based on ...,gpt-3.5-turbo,Weight stigma and health behaviors: evidence f...,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o...",Think about why this might be relevant for the...,Headline: Weight Stigma Associated with Poor H...


## New prompts

In [4]:
prep_step = [
    "Think about why this might be relevant for the audience in the grand scheme of things.\
    \nIdentify 1 or 2 key concepts from this article that would make interesting or helpful health content. \
    Exclude details that do not add value to the audience.\
    \nBased on the key concepts from the previous steps, extract the key points and numerical descriptors to",
]

summarize_task = [
    "summarize for a LinkedIn post.",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create an Instagram post without hashtags.",
]
edit_task = [
    "\nIf applicable, include a brief description of the research participants, such as age and sex.\
    Otherwise, you can skip this step.\
    \nEvaluate whether or not your writing may be confusing. \
    \nIf so, re-write it so it is clear. Otherwise, keep it the same. \
    \nCreate a journalistic headline to hook the audience.\
    \nReturn your response as the headline followed by the final version of the summary, \
    where the summary is in paragraph form.\
    \nDo not label the headline and summary, but separate them by a blank line.\
    \nDo not include a list of the key concepts as they should already be in the summary.",
]

system_role = "You are a journalist writing content based on science research articles."
prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
    columns=['prep_step', 'summarize_task', 'edit_task'])
prompts_df

,prep_step,summarize_task,edit_task
0,Think about why this might be relevant for the...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description o..."


## 1.2 new prompts

In [26]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, csv_path=folder_path, pickle_path=folder_path, json_path=folder_path
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature, system_role=system_role)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_initial',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[batch_summarize_chain()] Unable to save API response')

    return chaining_bot_dict

def create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    ):
    """
    Create DataFrame from initial ChatGPT summaries.
    """
    dfs_list = []
    chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        print(f'Processing {chatbot_key}...')
        # dfs_list.append(pd.DataFrame(chatbot_dict[chatbot_id][chatbot_key].qna).reset_index(names=['choice']))
        dfs_list.append(pd.DataFrame(
            chatbot_dict[chatbot_id][chatbot_key].qna, 
            index=[choice for choice in range(1, len(chatbot_dict[chatbot_id][chatbot_key].qna['summary'])+1)])
            )
    
    qna_df = pd.concat(dfs_list).reset_index(names=['choice'])
    # Move 'choice' to second column
    columns = qna_df.columns.tolist()
    columns.remove('choice')
    columns.insert(1, 'choice')
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    qna_dict[iteration_id] = qna_df[columns]
    return qna_dict

def spreadsheet_columns(qna_dict, chatbot_dict, iteration_id, chatbot_id=None,
    save=False, filename=None, csv_path=folder_path, pickle_path=folder_path
    ):
    """
    Update column names to include corresponding column in a spreadsheet (e.g. A, B, C)
    """
    qna_dict = create_qna_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
        )
    spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
    qna_dict[iteration_id].columns = [
        f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(qna_dict[iteration_id].columns)
        ]
    if save:
        description = filename if filename else 'batch_Chaining_summaries_initial'
        try:
            save_output(
                qna_dict[iteration_id], description=description, append_version=True,
                csv_path=csv_path, pickle_path=pickle_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[spreadsheet_columns()] Unable to save original summaries DataFrame')
    return qna_dict


# Set parameters
iteration_id = 1.2
n_choices = 2
pause_per_request=0
chatbot_id = iteration_id
save = True

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=True
    )

# chatbot_id = 1.15
# qna_dict = create_qna_df(
#     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
#     )
# chatbot_id = 1.14
# qna_dict = spreadsheet_columns(
#     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
#     )
# qna_dict[iteration_id]

**Text #1 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	n_previous_prompts
text2_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	n_previous_prompts
File saved:  ../text/2023-06-11/batch_Chaining_attributes_initial_2023-06-12_1308.sav
Time completed: 2023-06-12 13:08:58.970939
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00

In [31]:
chatbot_dict[iteration_id]

{'text1_prompt00': <__main__.Chaining at 0x17bd1aeb590>,
 'text2_prompt00': <__main__.Chaining at 0x17bce736f50>}

In [30]:
sample_Chaining_attr()

{'text': 'Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial\nDiscussion\nThis nutritional approach using high calcium and high protein dairy foods to increase calcium and protein intakes in institutionalised older adults replete in vitamin D was associated with a 33% reduction in risk of fractures of any type, a 46% reduction in risk of hip fractures, and an 11% reduction in risk of falls relative to controls. We found no group difference in all cause mortality.\nMost interventions aimed at reducing fracture risk target a drug therapy to people with osteoporosis because they are at high risk of fracture. This approach confers a large benefit to the individual and does so cost effectively, because few people need to be treated to avert one event. However, averting fractures in small numbers of people at high risk does not reduce the burden of fractures in the community.\nThe population burd

## Load response

In [9]:

filename = 'batch_Chaining_attributes_initial_2023-06-12_1308.sav'

loaded_pickle = loadpickle(filename, folder_path)
chatbot_dict[1.2] = revive_chatbot_dict(loaded_pickle)

Time completed: 2023-06-12 14:25:25.752201
Dictionary keys: ['text1_prompt00', 'text2_prompt00']
Article title: Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in residential care cluster randomised controlled trial
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'edit task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: []
	New chatbot attribute added: relevance_dict
		Attribute dictionary k

In [11]:
chatbot_dict[1.2].keys()

dict_keys(['text1_prompt00', 'text2_prompt00'])

## 1.21 strip white space; format CSV for easier copy/paste

In [ ]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        ):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False, csv_path=folder_path, pickle_path=folder_path, json_path=folder_path
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text, temperature=temperature, system_role=system_role)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_initial',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[batch_summarize_chain()] Unable to save API response')

    return chaining_bot_dict

def create_qna_df(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=None, 
    ):
    """
    Create DataFrame from initial ChatGPT summaries.
    """
    dfs_list = []
    chatbot_id = iteration_id if chatbot_id == None else chatbot_id
    for chatbot_key in chatbot_dict[chatbot_id].keys():
        print(f'Processing {chatbot_key}...')
        # dfs_list.append(pd.DataFrame(chatbot_dict[chatbot_id][chatbot_key].qna).reset_index(names=['choice']))
        dfs_list.append(pd.DataFrame(
            chatbot_dict[chatbot_id][chatbot_key].qna, 
            index=[choice for choice in range(1, len(chatbot_dict[chatbot_id][chatbot_key].qna['summary'])+1)])
            )
    
    qna_df = pd.concat(dfs_list).reset_index(names=['choice'])
    # Move 'choice' to second column
    columns = qna_df.columns.tolist()
    columns.remove('choice')
    columns.insert(1, 'choice')
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    qna_dict[iteration_id] = qna_df[columns]
    return qna_dict

def spreadsheet_columns(qna_dict, chatbot_dict, iteration_id, chatbot_id=None,
    save=False, filename=None, csv_path=folder_path, pickle_path=folder_path
    ):
    """
    Update column names to include corresponding column in a spreadsheet (e.g. A, B, C)
    """
    qna_dict = create_qna_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
        )
    spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
    qna_dict[iteration_id].columns = [
        f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(qna_dict[iteration_id].columns)
        ]
    if save:
        description = filename if filename else 'batch_Chaining_summaries_initial'
        try:
            save_output(
                qna_dict[iteration_id], description=description, append_version=True,
                csv_path=csv_path, pickle_path=pickle_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[spreadsheet_columns()] Unable to save original summaries DataFrame')
    return qna_dict


# Set parameters
iteration_id = 1.2
n_choices = 2
pause_per_request=0
chatbot_id = iteration_id
save = True

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=True
    )

# chatbot_id = 1.15
# qna_dict = create_qna_df(
#     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, 
#     )
# chatbot_id = 1.14
# qna_dict = spreadsheet_columns(
#     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
#     )
# qna_dict[iteration_id]

**Text #1 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	n_previous_prompts
text2_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	n_previous_prompts
File saved:  ../text/2023-06-11/batch_Chaining_attributes_initial_2023-06-12_1308.sav
Time completed: 2023-06-12 13:08:58.970939
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00

# *End of Page*